# 图构建

In [2]:
import dgl
import torch as th
# 边 0->1, 0->2, 0->3, 1->3
u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
g = dgl.graph((u, v))
print(g) # 图中节点的数量是DGL通过给定的图的边列表中最大的点ID推断所得出的（ID从0开始的）

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})


In [3]:
# 获取节点的ID
print(g.nodes())

tensor([0, 1, 2, 3])


In [4]:
# 获取边的对应端点
print(g.edges())

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))


In [5]:
# 获取边的对应端点和边ID
print(g.edges(form='all'))

(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))


In [7]:
# 如果具有最大ID的节点没有边，在创建图的时候，用户需要明确地指明节点的数量。
# 例如这里共计8个节点，4条边
g = dgl.graph((u, v), num_nodes=8)
print(g)

Graph(num_nodes=8, num_edges=4,
      ndata_schemes={}
      edata_schemes={})


## 转为无向图

无向图，即 A -> B, B -> A 均有一条边，相应地对应于原始的有向图，边的数目加倍

In [13]:
g = dgl.graph((u, v))

# 方法一： 自定义双向的“有向边”
u1 = th.cat((u, v), dim=0)
v1 = th.cat((v, u), dim=0)
bg1 = dgl.graph((u1, v1))
print(bg1)

# 方法二： 直接使用内置函数，转化成无向图
bg2 = dgl.to_bidirected(g)
print(bg2)

Graph(num_nodes=4, num_edges=8,
      ndata_schemes={}
      edata_schemes={})
Graph(num_nodes=4, num_edges=8,
      ndata_schemes={}
      edata_schemes={})


## 创建节点、边的特征

In [19]:
# 6个节点(0-5)，4条边
g = dgl.graph(([0, 0, 1, 5], [1, 2, 2, 0]))
print(g)

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={}
      edata_schemes={})


In [22]:
g.ndata['x'] = th.ones(g.num_nodes(), 3)               # 长度为3的节点特征, 1*3的全1向量
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)  # 标量整型特征
g

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32)})

In [24]:
# 不同名称的特征可以具有不同形状
g.ndata['y'] = th.randn(g.num_nodes(), 5)
g

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32)})

In [25]:
print(g.ndata['x'][1])                  # 获取节点1的特征
print(g.edata['x'][th.tensor([0, 3])])  # 获取边0和3的特征

tensor([1., 1., 1.])
tensor([1, 1], dtype=torch.int32)


In [26]:
# 加权图，将边的权重放入“w”特征之中
weight = th.tensor([0.1, 0.6, 0.9, 0.5])
g.edata['w'] = weight
g

Graph(num_nodes=6, num_edges=4,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32), 'y': Scheme(shape=(5,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(), dtype=torch.int32), 'w': Scheme(shape=(), dtype=torch.float32)})

基于以上，我们已经初步建立好了图的存储，包括有向图，无向图，带权图

## 异构图

In [28]:
# 创建一个具有2种节点类型和2种边类型的异构图
graph_data = {
   ('user', 'follows', 'user'): (th.tensor([0, 1]), th.tensor([1, 2])), # 用户关注用户
   ('user', 'plays', 'game'): (th.tensor([0, 1]), th.tensor([2, 3]))    # 用户玩游戏
}
# 跟签名吗一样，nnodes和nedges都是 节点ID，边ID的最大值+1
hg = dgl.heterograph(graph_data)
print(hg)
print(hg.ntypes)
print(hg.etypes)
print(hg.canonical_etypes)

Graph(num_nodes={'game': 4, 'user': 3},
      num_edges={('user', 'follows', 'user'): 2, ('user', 'plays', 'game'): 2},
      metagraph=[('user', 'user', 'follows'), ('user', 'game', 'plays')])
['game', 'user']
['follows', 'plays']
[('user', 'follows', 'user'), ('user', 'plays', 'game')]


In [29]:
# 获取图中所有节点的数量
print(g.num_nodes())
# 获取user节点的数量
print(g.num_nodes('user'))
# 不同类型的节点有单独的ID。因此，没有指定节点类型就没有明确的返回值。

print(g.nodes('user'))

7
3
tensor([0, 1, 2])


In [30]:
# 如果有多种类型的边、节点，获取时，必须指定类型，否则就会出现如下错误!!
g.nodes()

DGLError: Node type name must be specified if there are more than one node types.